## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
import json
sys.path.append('../..')
import stober_synthesis_utils as stober
import usaxs_utils
import uuid

In [3]:
with open('systemconfig.json', 'rt') as f:
    config = json.load(f)

#usaxs_server_ip = config['sample_server']['ip_address']

#afl_ip = '192.168.1.4'

reaction_time = 5*60

In [4]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = 'Mesoporous_timegrowth_5_22_25.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [5]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
response in connect:  [True, True, True, True]


In [6]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [7]:
sample_table = pd.read_csv('2025_04_15_MesoporousRepeatabilitySampleTable.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [12]:
sample_table = sample_table.iloc[5:8]

## Load Tools

In [13]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}
Syringe name:  1cc_4
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_4'}


In [43]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

In [16]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [17]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [80]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [74]:
water_syringe.load_syringe(0, water_syringe.empty_position - 1)

Loaded syringe, remaining volume 0 uL


## Load ammonia syringe

In [21]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [20]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [81]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [24]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [25]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position + 200, s = 200)

In [26]:
ammonia_syringe.load_syringe(550, ammonia_syringe.full_position + 200)

Loaded syringe, remaining volume 550 uL


## Load TEOS syringe

In [27]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [82]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [29]:
teos_syringe.load_syringe(0, teos_syringe.empty_position - 1)

Loaded syringe, remaining volume 0 uL


## load ethanol syringe

In [33]:
#jubilee.park_tool()

In [30]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [31]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [75]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load labware


In [36]:
samples_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples_1.manual_offset([(30.8, 168.0), (121.3, 168.3), (121.3, 112.6)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [37]:
stocks_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_1.manual_offset([(30.1, 264.5), (121.1, 264.5), (121.1, 208.5)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [38]:
stocks_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_2.manual_offset([(171.1, 264.8), (260.8, 265.0), (260.8, 209.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [78]:
samples = stober.SampleWells({'samples_1':samples_1})

In [40]:
teos_stocks = stober.Stocks([stocks_1[0]], usable_volume= 17000)
ammonia_stocks = stober.Stocks([stocks_1[1]], usable_volume= 17000)
water_stocks = stober.Stocks([stocks_1[2]], usable_volume= 17000)
ethanol_stocks = stober.Stocks([stocks_1[3], stocks_1[4], stocks_1[5]], usable_volume = 17000)
f127_stocks = stober.Stocks([stocks_1[6]], usable_volume = 17000)
ctab_stocks = stober.Stocks([stocks_1[7]], usable_volume= 17000)

rinse_stocks_precursor = [stocks_1[9], stocks_1[10], stocks_1[11]]
rinse_stocks_teos = [stocks_2[0], stocks_2[1], stocks_2[2], stocks_2[3]]

## Check sample alignment

In [41]:
jubilee.deck.safe_z

68.5

In [44]:
jubilee.pickup_tool(mix_syringe)
for loc in samples_1:
    jubilee.safe_z_movement()
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = loc
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

168.0


Is the tip centered over sample Well A1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2? 


168.05000006867903


Is the tip centered over sample Well A2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2? 


168.10000013735805


...

Is the tip centered over sample Well C2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2? 


112.40007664543361


Is the tip centered over sample Well C3 form 20mLscintillation 12 Well Plate 18000 µL on slot 2? 


112.45007671411264


Is the tip centered over sample Well C4 form 20mLscintillation 12 Well Plate 18000 µL on slot 2? 


In [45]:
jubilee.park_tool()

In [46]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume,Unnamed: 8,Unnamed: 9
5,SyringeMixControl_1,d3092f76-88a7-4ccb-82c1-5cc7b8356c68,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN
6,SyringeMixControl_2,30d3b674-6fa8-4c98-b282-da7523f58a0f,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN
7,SyringeMixControl_3,4039d278-a8ec-4eac-a4ca-bd5ee2be894d,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN


## Full synthesis

In [62]:
samples.wells

{'samples_1_A1': {'well': Well A1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'used',
  'sample': 'd3092f76-88a7-4ccb-82c1-5cc7b8356c68'},
 'samples_1_A2': {'well': Well A2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'used',
  'sample': 'd3092f76-88a7-4ccb-82c1-5cc7b8356c68'},
 'samples_1_A3': {'well': Well A3 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'used',
  'sample': '30d3b674-6fa8-4c98-b282-da7523f58a0f'},
 'samples_1_A4': {'well': Well A4 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'clean',
  'sample': None},
 'samples_1_B1': {'well': Well B1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'clean',
  'sample': None},
 'samples_1_B2': {'well': Well B2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'clean',
  'sample': None},
 'samples_1_B3': {'well': Well B3 form 20mLscintillation 12 Well Plate 18000 µL on slot 2,
  'status': 'clean',
  '

In [72]:
jubilee.park_tool()

In [79]:
j = 0
for i, row in sample_table.iterrows():


    #if i % 2 == 0 and i !=0:
    #    # swap out CTAB
    #    result = input('CTAB swap out required. swap ctab with fresh from the hot plate.')
    #else:
    #    r = input('Please home Jubilee')
        
    sample_uid = row['uuid']
    sample_name = row['sample_name']
    well = samples.get_well_for_sample(sample_uid)
    teos_volume = row['teos_volume']
    ethanol_volume = row['ethanol_volume']
    ctab_volume = row['ctab_volume']
    f127_volume = row['F127_volume']

    sample_table_select = sample_table.iloc[j:j+1]
    location_lookup_selected = {sample_uid:well}

    print(sample_table_select)



    start_time = time.time()
    logger.info(f'Starting sample {sample_name}, {sample_uid} in well {well}')
    print(f'Starting sample {sample_name}, {sample_uid} in well {well}')
    #add ethanol
    stober.reactant_transfer(jubilee, mix_syringe, ethanol_stocks, well, ethanol_volume, 500)
    logger.info(f'Added {ethanol_volume} uL ethanol to {str(sample_uid)}') 
    
    #add water
    stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'water_volume', water_stocks, dwell_time = 7)
    print(f'added water')
    logger.info(f'Added water to {str(sample_uid)}') 
    #add ammonia
    stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'ammonia_volume', ammonia_stocks, dwell_time = 10, refill_dwell=5)
    print('added ammonia')
    logger.info(f'Added ammonia to {str(sample_uid)}') 
    # add F127
    
    stober.reactant_transfer(jubilee, surfactant_syringe, f127_stocks, well, f127_volume, 50, rinse_stocks=rinse_stocks_precursor)
    print(f'added {f127_volume} f127')
    logger.info(f'Added {f127_volume} uL F127 to {str(sample_uid)}') 
    
    stober.reactant_transfer(jubilee, surfactant_syringe, ctab_stocks, well, ctab_volume, 50, rinse_stocks=rinse_stocks_precursor)
    print(f'added {ctab_volume} uL CTAB')
    logger.info(f'Added {ctab_volume} uL ctab to {str(sample_uid)}') 
    # mix precursors
    
    stober.first_mix(jubilee, mix_syringe, 9000, location_lookup_selected, rinse_stocks_precursor, 5, n_rinse = 3)
    print('mixed samples')
    # add teos
    teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'teos_volume', teos_stocks, mix_after=(9000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)
    
    logger.info(f'Added TEOS to sample {sample_name} at {time.strftime("%H:%M", time.localtime(teos_add_time))}')
    
    print(f'Time from start to TEOS addition: {teos_add_time - start_time} s')
    
    print(f'########## Complete sample {sample_name} ############')
    i#nput(f'Sample {sample_name} complete, please add TEOS')

    j+=1
    _ = input('Remove sample')




           sample_name                                  uuid  teos_volume  \
5  SyringeMixControl_1  d3092f76-88a7-4ccb-82c1-5cc7b8356c68   236.892048   

   ammonia_volume  water_volume  ethanol_volume  ctab_volume  F127_volume  \
5      180.111534   1313.594844     3546.832137  3351.899498  1370.669939   

   Unnamed: 8  Unnamed: 9  
5         NaN         NaN  
Starting sample SyringeMixControl_1, d3092f76-88a7-4ccb-82c1-5cc7b8356c68 in well Well A1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2
Loaded syringe, remaining volume 0 uL
breaking dispense into 1 of volume 3546.8321374196094
Loaded syringe, remaining volume 0 uL
breaking dispense into 2 of volume 656.797422034821
remaining vol:  949.0
remaining vol:  949.0
added water
breaking dispense into 1 of volume 180.1115338690579
added ammonia
Loaded syringe, remaining volume 0 uL
breaking dispense into 2 of volume 685.3349693119526
Rinsing 1cc_4
Loaded syringe, remaining volume 0 uL
added 1370.6699386239052 f127
Loaded syr

KeyboardInterrupt: Interrupted by user

In [66]:
sample_table_select

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume,Unnamed: 8,Unnamed: 9
6,SyringeMixControl_2,30d3b674-6fa8-4c98-b282-da7523f58a0f,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN


In [67]:
sample_uid

'd3092f76-88a7-4ccb-82c1-5cc7b8356c68'

In [68]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume,Unnamed: 8,Unnamed: 9
5,SyringeMixControl_1,d3092f76-88a7-4ccb-82c1-5cc7b8356c68,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN
6,SyringeMixControl_2,30d3b674-6fa8-4c98-b282-da7523f58a0f,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN
7,SyringeMixControl_3,4039d278-a8ec-4eac-a4ca-bd5ee2be894d,236.892048,180.111534,1313.594844,3546.832137,3351.899498,1370.669939,NaN,NaN


In [51]:
sample_table_select['water_volume']

Series([], Name: water_volume, dtype: float64)

In [171]:
jubilee.park_tool()

In [58]:
sample_table.loc[5]

sample_name                        SyringeMixControl_1
uuid              d3092f76-88a7-4ccb-82c1-5cc7b8356c68
teos_volume                                 236.892048
ammonia_volume                              180.111534
water_volume                               1313.594844
ethanol_volume                             3546.832137
ctab_volume                                3351.899498
F127_volume                                1370.669939
Unnamed: 8                                         NaN
Unnamed: 9                                         NaN
Name: 5, dtype: object

In [53]:
i

5